In [1]:
import sqlite3 as sl

db = sl.connect('server.db')
cursor = db.cursor()

In [2]:
cursor.execute("""
    SELECT (?) FROM acc
""", ('id', 'login')).fetchall()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 1, and there are 2 supplied.

In [2]:
cursor.execute("""
    SELECT * FROM acc
""").fetchall()

[(1,
  'testlogin',
  'qwerty',
  None,
  'test',
  'test@mail.ru',
  '11.11.2011',
  'test',
  'HUI',
  None,
  None),
 (2,
  'testlogin1',
  'qwerty1',
  None,
  'test1',
  'test1@mail.ru',
  '11.11.2011',
  'test',
  'HUI',
  None,
  None)]

In [41]:
cursor.execute(
    """
    INSERT INTO acc(login2, pass, username, mail, reg_date, first_name, last_name)
    VALUES 
        ('testlogin1', 'qwerty1', 'test1', 'test1@mail.ru', '11.11.2011', 'test', 'HUI'),
        ('testlogin2', 'qwerty2', 'test2', 'test2@mail.ru', '11.11.2011', 'test', 'HUI')
    """
)
db.commit()

OperationalError: table acc has no column named login2

In [17]:
usernames = ['test', 'test1']

In [17]:
from pydantic import BaseModel
from typing import *
class User(BaseModel):
    login: str
    passw: str
    api: Optional[str] = None
    username: str
    mail: str
    reg_date: str
    first_name: str
    last_name: str
    graduation: Optional[str] = None
    company: Optional[str] = None

class UserIn(BaseModel):
    login: Optional[str] = None
    username: Optional[str] = None
    mail: Optional[str] = None

class UserOut(BaseModel):
    api: Optional[str] = None
    username: str
    mail: str
    first_name: str
    last_name: str
    graduation: Optional[str] = None
    company: Optional[str] = None

In [18]:
user = UserOut(username='a', mail='b', first_name='c', last_name='d')

In [27]:
cols = ', '.join(user.dict(exclude_defaults=True).keys())
vals = user.dict(exclude_defaults=True).values()

In [28]:
len(vals)

4

In [31]:
query = """INSERT INTO acc({}) VALUES({})"""
query.format(cols, ', '.join(['?']* len(vals)))

'INSERT INTO acc(username, mail, first_name, last_name) VALUES(?, ?, ?, ?)'

In [20]:
cols

'username, mail, first_name, last_name'

In [21]:
vals

'a, b, c, d'

In [12]:
', '.join(u.dict().values())

TypeError: sequence item 0: expected str instance, NoneType found

In [5]:
', '.join(UserOut.__fields__)

'api, username, mail, first_name, last_name, graduation, company'

In [52]:
u = UserIn(login='1234')

In [68]:
tuple(u.dict(skip_defaults=True).items())[0]

C:\Users\kozlo\AppData\Local\Temp\ipykernel_23220\2364136110.py:1: DeprecationWarning: UserIn.dict(): "skip_defaults" is deprecated and replaced by "exclude_unset"
  tuple(u.dict(skip_defaults=True).items())[0]


('login', '1234')

In [54]:
a, v = u.dict().items()
print(a, v)

ValueError: too many values to unpack (expected 2)

In [40]:
try:
    cursor.execute(f'SELECT * FROM acc WHERE ids = 1')
except sl.OperationalError as e:
    raise e

OperationalError: no such column: ids

In [42]:
acc_columns = [
    i[1] 
    for i in cursor.execute("PRAGMA table_info(acc)").fetchall()
]

In [43]:
dict(
    zip(
        acc_columns,
        cursor.execute(f'SELECT * FROM acc WHERE id = 1').fetchone()
    )
)

{'id': 1,
 'login': 'testlogin',
 'pass': 'qwerty',
 'api': None,
 'username': 'test',
 'mail': 'test@mail.ru',
 'reg_date': '11.11.2011',
 'first_name': 'test',
 'last_name': 'HUI',
 'graduation': None,
 'company': None}

In [33]:
[cursor.execute(f'SELECT * FROM acc WHERE id = {i}').fetchone() for i in (1, 3)]

[(1,
  'testlogin',
  'qwerty',
  None,
  'test',
  'test@mail.ru',
  '11.11.2011',
  'test',
  'HUI',
  None,
  None),
 None]

In [37]:
[i[1] for i in cursor.execute("PRAGMA table_info(acc)").fetchall()]

['id',
 'login',
 'pass',
 'api',
 'username',
 'mail',
 'reg_date',
 'first_name',
 'last_name',
 'graduation',
 'company']

In [22]:
[cursor.execute(f'SELECT * FROM acc WHERE username = ?;', (i,)).fetchone() for i in usernames]

[(1,
  'testlogin',
  'qwerty',
  None,
  'test',
  'test@mail.ru',
  '11.11.2011',
  'test',
  'HUI',
  None,
  None),
 (2,
  'testlogin1',
  'qwerty1',
  None,
  'test1',
  'test1@mail.ru',
  '11.11.2011',
  'test',
  'HUI',
  None,
  None)]

In [23]:
query = """
    INSERT INTO acc(login, pass, api, username, mail, reg_date, first_name, last_name, graduation, company)
    VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""